## loading the data file of least 50 labels
* it has video caption, start, end segments for each videos of each labels

In [156]:
import json
with open("/home/grads/h/hasnat.md.abdullah/open_ended_activity_analysis/zero-shot-video-to-text/data/least_50_label_video_id_segment_caps.json","r") as f:
    least_50_label_video_id_segment_caps = json.load(f)



## extracting the all segments of first video of each label
* extract all the videos of each label
* extract all the segments of first video of each label

In [157]:
video_details = []


for label in least_50_label_video_id_segment_caps.keys():
    # print(label)
    
    # print("len(least_50_label_video_id_segment_caps[label].keys()):", len(least_50_label_video_id_segment_caps[label].keys()))
    for video_id in least_50_label_video_id_segment_caps[label].keys():
        video = least_50_label_video_id_segment_caps[label][video_id]
        # print("id:", video_id)
        # print(video)
        # print(video[0])
        for i in video: 

            video_details.append(i)
        break


    # break
video_details

[{'caption': 'A furniture renovation company advertises their services by showing several homes and tools they have worked on and use and what type of work they do.',
  'start': 0.0,
  'end': 50.48,
  'path': 'Polishing_forniture/Ou24uqaFRPg.mp4'},
 {'caption': 'A man in an apron carefully puts a colored substance on a plate.',
  'start': 9.84,
  'end': 12.87,
  'path': 'Polishing_forniture/Ou24uqaFRPg.mp4'},
 {'caption': 'The man mixes two of the different colored substances with a paint brush.',
  'start': 15.4,
  'end': 17.42,
  'path': 'Polishing_forniture/Ou24uqaFRPg.mp4'},
 {'caption': 'The man carefully touches the paint brush to an old table coloring portions of the table that have been damaged.',
  'start': 17.67,
  'end': 25.49,
  'path': 'Polishing_forniture/Ou24uqaFRPg.mp4'},
 {'caption': 'A different man in an apron and face mask sands an old table.',
  'start': 25.75,
  'end': 37.1,
  'path': 'Polishing_forniture/Ou24uqaFRPg.mp4'},
 {'caption': 'The man then puts polish o

In [158]:
len(video_details)

249

In [159]:
with open("least_50_activity_net_first_vid_all_seg.json",'w') as f: 
    json.dump(video_details,f,indent=4)

In [160]:
video_details[7]

{'caption': 'Information on how to reach the restoration company is displayed.',
 'start': 41.65,
 'end': 50.48,
 'path': 'Polishing_forniture/Ou24uqaFRPg.mp4'}

## running zero-shot captioner on video_details data

In [170]:
video_details[154]

{'caption': 'The female then uses the rag to rub the inside of the sink.',
 'start': 24.05,
 'end': 50.39,
 'path': 'Cleaning_sink/-5h2E52tl4Q.mp4'}

In [175]:
output_path = "/home/grads/h/hasnat.md.abdullah/open_ended_activity_analysis/zero-shot-video-to-text/output_least50_first_vid_all_seg.json"
with open(output_path,'r') as f:
    testing = json.load(f)

testing[153]

{'caption': 'The female opens a container of cleaner and puts it on a rag.',
 'start': 12.02,
 'end': 23.48,
 'path': 'Cleaning_sink/-5h2E52tl4Q.mp4'}

In [190]:
import os
import subprocess
from tqdm import tqdm
# videos_path = "/home/grads/h/hasnat.md.abdullah/open_ended_activity_analysis/zero-shot-video-to-text/least_50_activity_net_first_vid_all_seg.json"
# with open(videos_path,'r') as f:
#     video_details = json.load(f)

# testing = video_details
video_dataset_dir = "data/ActivityNet_200/training/"


output_path = "/home/grads/h/hasnat.md.abdullah/open_ended_activity_analysis/zero-shot-video-to-text/output_least50_first_vid_all_seg.json"
with open(output_path,'r') as f:
    testing = json.load(f)
# print(testing)
id=153
for id in tqdm(range(id,len(testing))) :
    # print(id)
    # print(data)
    data = testing[id]
    # print(data)
    path = video_dataset_dir+ str(data['path'])
    start = data['start']
    end = data['end']

    # print(path)
    # print(start)
    # print(end)
    # Construct the CLI command with the extracted values
    if os.path.exists(path):
        command = f"python run.py --token_wise --randomized_prompt --run_type caption_videos --data_path {path} --start_sec {start} --end_sec {end}"
        # Run the command and capture the output
        output = subprocess.check_output(command, shell=True, universal_newlines=True)
        generated_caption = output.split("\n")[-3].split(":")[1].strip()
        testing[id]["generated_cap"] = generated_caption
    else:
        testing[id]["generated_cap"] = "_"
    
    if id%2==0:
        with open("output_least50_first_vid_all_seg.json","w") as f: 
            json.dump(testing,f,indent=4)
    # break
    
    
    




100%|██████████| 96/96 [2:56:35<00:00, 110.37s/it]  


In [192]:
testing[0]

{'caption': 'A furniture renovation company advertises their services by showing several homes and tools they have worked on and use and what type of work they do.',
 'start': 0.0,
 'end': 50.48,
 'path': 'Polishing_forniture/Ou24uqaFRPg.mp4',
 'generated_cap': 'Video depicting the kitchen table in a wood carving is making rounds.'}

In [216]:
import statistics
import nltk
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
reference_captions = []
candidate_captions = []
for i in testing: 
    if i['generated_cap'] != "_":
        
        reference_captions.append(i['caption'])
        candidate_captions.append(i['generated_cap'])
# reference_captions
# Tokenize the captions
reference_captions = [caption.split() for caption in reference_captions]
candidate_captions = [caption.split() for caption in candidate_captions]

# print(reference_captions)
# Calculate BLEU score


individual_bleu_scores = []

for n in range(1, 5):
    bleu_scores = [sentence_bleu(ref, cand, weights=(1.0 / n,)) for ref, cand in zip(reference_captions, candidate_captions)]
    # print(type(bleu_scores))
    average_bleu_score = statistics.mean(bleu_scores)
    individual_bleu_scores.append(average_bleu_score)

# Print individual BLEU scores
print("Individual BLEU scores:")
for i, score in enumerate(individual_bleu_scores):
    print(f"BLEU@{i+1}: {score:.4f}")

Individual BLEU scores:
BLEU@1: 0.0555
BLEU@2: 0.2009
BLEU@3: 0.3096
BLEU@4: 0.3846


In [1]:
import json
import statistics
import nltk
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
reference_captions = []
candidate_captions = []
with open ("/home/grads/h/hasnat.md.abdullah/open_ended_activity_analysis/zero-shot-video-to-text/output_least50_first_vid_all_seg.json",'r') as f:
    testing = json.load(f)
for i in testing: 
    if i['generated_cap'] != "_":
        
        reference_captions.append(i['caption'])
        candidate_captions.append(i['generated_cap'])
# reference_captions
# Tokenize the captions
reference_captions = [caption.split() for caption in reference_captions]
candidate_captions = [caption.split() for caption in candidate_captions]

# print(reference_captions)
# Calculate BLEU score


cumulative_bleu_scores = []
weight_list = [(1, 0, 0, 0),(0.5, 0.5, 0, 0),(0.33, 0.33, 0.33, 0),(0.25, 0.25, 0.25, 0.25)]
for n in range(1, 5):
    bleu_scores = [sentence_bleu(ref, cand, weights=weight_list[n-1]) for ref, cand in zip(reference_captions, candidate_captions)]
    
    # print(type(bleu_scores))

    average_bleu_score = statistics.mean(bleu_scores)
    cumulative_bleu_scores.append(average_bleu_score)

# Print cumulative BLEU scores
print("cumulative BLEU scores:")
for i, score in enumerate(cumulative_bleu_scores):
    print(f"BLEU@{i+1}: {score:.4f}")

/home/grads/h/hasnat.md.abdullah/open_ended_activity_analysis/oeaa/lib/python3.6/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/grads/h/hasnat.md.abdullah/open_ended_activity_analysis/oeaa/lib/python3.6/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/grads/h/hasnat.md.abdullah/open_ended_activity_analysis/oeaa/lib/python3.6/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score

cumulative BLEU scores:
BLEU@1: 0.0555
BLEU@2: 0.0000
BLEU@3: 0.0000
BLEU@4: 0.0000
